In [1]:
import bs4
import time
from datetime import datetime
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

In [2]:
def get_article_html(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content,"html.parser")
    time.sleep(30) 
    return soup

In [3]:
def get_data_moneycontrol_web(soup):
    all_articles = soup.find('ul',attrs={'id':'cagetory'}).find_all('li',attrs={'class':'clearfix'})
    
    # Declaration
    articles_topic = []
    articles_content = []
    
    for article in all_articles:
            topic = article.find('h2').find('a').get_text()
            
            url_of_article = article.find('h2').find('a')['href']
            soup = get_article_html(url_of_article)
            
            if soup.find('div',attrs={'class':'content_wrapper arti-flow'}) != None:
                content = soup.find('div',attrs={'class':'content_wrapper arti-flow'}).find_all("p")
                full_content = ""
                if content != None:
                    for paragraph in content:
                        full_content = full_content + " " + paragraph.get_text().strip()
                        
                articles_topic.append(topic)
                articles_content.append(full_content)            

    return articles_topic, articles_content

In [4]:
def insert_data_to_csv(topic, category, content):
    # Create dataframe
    data = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    data.to_csv(f'moneycontrol Scraping-{category}-{datetime.now().strftime("%d-%m-%Y")}.csv', mode='a')

In [ ]:
categories = ["markets", "stocks", "companies", "trends", "business", "economy"]

print("Start scraping from moneycontrol!")

def scraping_with_selenium_from_url(url):
        driver = webdriver.Chrome()
        driver.get(url)
        time.sleep(30)
        driver.maximize_window()
        time.sleep(30)
        html = driver.page_source
        soup = BeautifulSoup(html,"html.parser")
        topic, content = get_data_moneycontrol_web(soup)
        insert_data_to_csv(topic, category, content)
        next_button = driver.find_element(by=By.LINK_TEXT, value=('»')).click()
        for i in range (1, 28): 
            time.sleep(30)
            html = driver.page_source
            soup = BeautifulSoup(html,"html.parser")
            topic, content = get_data_moneycontrol_web(soup)
            insert_data_to_csv(topic, category, content)
            next_button = driver.find_element(by=By.LINK_TEXT, value=('»')).click()
        driver.close()
        
for category in categories:
    if category in ("economy", "markets", "stocks", "companies"):
        scraping_with_selenium_from_url(f"https://www.moneycontrol.com/news/business/{category}")
    else:
        scraping_with_selenium_from_url(f"https://www.moneycontrol.com/news/{category}")

print("Finish scraping from moneycontrol.com!")

Start scraping from moneycontrol!
